白井さんコードを元に，渋谷を出発地とするようなトリップを抽出
df_diaryにODの緯度経度を紐づけるために必要


In [46]:
import pandas as pd 
import numpy as np 

In [47]:
df_trip = pd.read_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/all_trip.csv',index_col = 0)
# diaryを読み込む
# diaryデータにocarもdcarも紐づけてから読み込んだほうがいいかもね．
df_diary = pd.read_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/diary_cleaned.csv')
# loc = pd.read_csv('/Users/honokashirai/Downloads/Shibuya/データ整形/all_trip.csv',index_col = 0)

In [48]:
print(df_diary)

      Unnamed: 0   id    day         time  purpose    d    o  d_line  d_car  \
0              0   26  12/19  10:00-11:00      2.0  7.0  5.0     1.0    4.0   
1              1   20  12/21  13:00-14:00     11.0  NaN  NaN    10.0    NaN   
2              2   34  12/20  14:00-15:00      NaN  7.0  7.0     1.0    3.0   
3              3  244  12/21  18:00-19:00      4.0  NaN  NaN     1.0    4.0   
4              4  120  12/19    6:00-7:00      2.0  NaN  NaN     1.0    1.0   
...          ...  ...    ...          ...      ...  ...  ...     ...    ...   
1574        1574   78  01/28  21:00-22:00      4.0  1.0  1.0     1.0    8.0   
1575        1575  179  01/27  17:00-18:00     11.0  NaN  NaN     1.0    8.0   
1576        1576  105  01/30    8:00-9:00     11.0  NaN  NaN     9.0    NaN   
1577        1577   73  01/27  18:00-19:00      4.0  1.0  1.0     4.0    3.0   
1578        1578   84  01/29  19:00-20:00     11.0  NaN  NaN     6.0    NaN   

     staire_to_gate  d_gate  o_line  o_gate staire_

In [49]:
# id列を整える
# df_diary['id'] = str(df_diary['id'])
# df_diary['id'] = df_diary['id'].apply(lambda x: f"23{x.zfill(3)}").astype(int)
df_diary['id'] = df_diary['id'].apply(lambda x: 23000 + x).astype(int)

def convert_date(date_str):
    # 日付部分を抽出（最初の "/" までの部分とその後の部分に分ける）
    # date_part = date_str.split('（')[0]
    
    # 月と日を分割
    month, day = map(int, date_str.split('/')) # map関数でtupleに
    
    # 年を決定
    if month == 12:
        year = 2022
    else:
        year = 2023
    
    # ISOフォーマットに変換
    return f"{year:04d}-{month:02d}-{day:02d}"

df_diary['day'] = df_diary['day'].apply(convert_date)

from datetime import datetime
def convert_datetime_to_date(datetime_str):
    datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    return datetime_obj.strftime('%Y-%m-%d')

def convert_str_time(time_str): # 
    a = time_str.split(':')[0]
    return int(a)




In [50]:
print(df_diary)

      Unnamed: 0     id         day         time  purpose    d    o  d_line  \
0              0  23026  2022-12-19  10:00-11:00      2.0  7.0  5.0     1.0   
1              1  23020  2022-12-21  13:00-14:00     11.0  NaN  NaN    10.0   
2              2  23034  2022-12-20  14:00-15:00      NaN  7.0  7.0     1.0   
3              3  23244  2022-12-21  18:00-19:00      4.0  NaN  NaN     1.0   
4              4  23120  2022-12-19    6:00-7:00      2.0  NaN  NaN     1.0   
...          ...    ...         ...          ...      ...  ...  ...     ...   
1574        1574  23078  2023-01-28  21:00-22:00      4.0  1.0  1.0     1.0   
1575        1575  23179  2023-01-27  17:00-18:00     11.0  NaN  NaN     1.0   
1576        1576  23105  2023-01-30    8:00-9:00     11.0  NaN  NaN     9.0   
1577        1577  23073  2023-01-27  18:00-19:00      4.0  1.0  1.0     4.0   
1578        1578  23084  2023-01-29  19:00-20:00     11.0  NaN  NaN     6.0   

      d_car staire_to_gate  d_gate  o_line  o_gate 

In [51]:
df_diary['arr_time'] = df_diary['time'].apply(convert_str_time) # これは着いた時刻の一番最初の時間hourを出している
print(df_diary['arr_time']) 

0       10
1       13
2       14
3       18
4        6
        ..
1574    21
1575    17
1576     8
1577    18
1578    19
Name: arr_time, Length: 1579, dtype: int64


出発地が駅まちのtripに着目する場合は出発時刻と出発地緯度経度に着目する

In [52]:
df_trip['日付'] = df_trip['出発時刻'].apply(convert_datetime_to_date)
df_trip['出発時刻'] = pd.to_datetime(df_trip['出発時刻'])
df_trip['時間'] = df_trip['出発時刻'].dt.hour

In [53]:
df_trip = df_trip[['ユーザーID','日付', '時間', '出発地緯度', '出発地経度', 'O_cell']]
df_trip = df_trip[df_trip['O_cell']==1]
print(df_trip)

      ユーザーID          日付  時間      出発地緯度       出発地経度  O_cell
5404   23001  2023-01-09   8  35.657482  139.702622       1
3466   23001  2023-01-10   7  35.656372  139.702808       1
521    23002  2022-12-19  19  35.658111  139.698929       1
520    23002  2022-12-19  19  35.657393  139.697121       1
1214   23002  2022-12-20  19  35.660637  139.700777       1
...      ...         ...  ..        ...         ...     ...
9      23259  2023-01-11  18  35.658119  139.702204       1
20     23259  2023-01-27  14  35.659396  139.700047       1
122    23259  2023-01-30  19  35.659040  139.700447       1
57     23260  2023-01-28   9  35.659154  139.702222       1
7      23260  2023-01-29  11  35.658713  139.701957       1

[1079 rows x 6 columns]


In [54]:
df_diary['id'] = df_diary['id'].astype(int)
df_trip['ユーザーID'] = df_trip['ユーザーID'].astype(int)

df_diary = pd.merge(df_diary, df_trip, right_on = ['ユーザーID', '日付'], left_on = ['id', 'day'], how = 'inner')
#df_trip['abs_diff'] = (df['時間'] - df['arr_time']).abs()
print(df_diary)


     Unnamed: 0     id         day         time  purpose    d    o  d_line  \
0             0  23026  2022-12-19  10:00-11:00      2.0  7.0  5.0     1.0   
1             2  23034  2022-12-20  14:00-15:00      NaN  7.0  7.0     1.0   
2             3  23244  2022-12-21  18:00-19:00      4.0  NaN  NaN     1.0   
3             5  23005  2022-12-22  17:00-18:00      2.0  1.0  1.0     8.0   
4             8  23231  2022-12-20  13:00-14:00     11.0  NaN  NaN     2.0   
..          ...    ...         ...          ...      ...  ...  ...     ...   
886        1548  23195  2023-01-29  13:00-14:00      5.0  NaN  5.0     2.0   
887        1548  23195  2023-01-29  13:00-14:00      5.0  NaN  5.0     2.0   
888        1552  23156  2023-01-30    8:00-9:00      1.0  NaN  NaN     1.0   
889        1560  23040  2023-01-29  12:00-13:00      8.0  NaN  2.0     1.0   
890        1561  23171  2023-01-30  17:00-18:00      8.0  5.0  1.0     2.0   

     d_car staire_to_gate  ...  o_gate  staire_to_home  o_car a

In [55]:
df_diary['abs_diff'] = (df_diary['時間'] - df_diary['arr_time']).abs()

df_filtered = df_diary[df_diary['abs_diff'] <= 2].drop(columns=['abs_diff'])
# df_filtered = df_filtered[['トリップID', 'アプリのID', '渋谷駅を訪れた日付を教えてください．', '渋谷駅を訪れた時間帯を教えてください．', '到着地緯度', '到着地経度']]


In [56]:
print(df_filtered.columns)

Index(['Unnamed: 0', 'id', 'day', 'time', 'purpose', 'd', 'o', 'd_line',
       'd_car', 'staire_to_gate', 'd_gate', 'o_line', 'o_gate',
       'staire_to_home', 'o_car', 'arr_time', 'ユーザーID', '日付', '時間', '出発地緯度',
       '出発地経度', 'O_cell'],
      dtype='object')


In [57]:
df_filtered = df_filtered[['id', 'day', 'time', 'purpose', '出発地緯度', '出発地経度']]



In [58]:
df_filtered

,id,day,time,purpose,出発地緯度,出発地経度
0,23026,2022-12-19,10:00-11:00,2.0,35.659624,139.699793
1,23034,2022-12-20,14:00-15:00,NaN,35.661194,139.698300
2,23244,2022-12-21,18:00-19:00,4.0,35.659481,139.701597
3,23005,2022-12-22,17:00-18:00,2.0,35.657916,139.701920
4,23231,2022-12-20,13:00-14:00,11.0,35.658515,139.701615
...,...,...,...,...,...,...
886,23195,2023-01-29,13:00-14:00,5.0,35.663147,139.700730
887,23195,2023-01-29,13:00-14:00,5.0,35.658291,139.698335
888,23156,2023-01-30,8:00-9:00,1.0,35.653565,139.692438
889,23040,2023-01-29,12:00-13:00,8.0,35.657413,139.706326


In [59]:
df_filtered.to_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/gate_MNL/filtered_shibu_o_loc.csv')